In [8]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Define the color labels
colors = ['black', 'blue', 'brown', 'gray', 'green', 'orange', 'purple', 'red', 'white', 'yellow']

# Get the current working directory
current_dir = os.getcwd()

# Paths to your directories (navigate up two levels from current directory)
relative_train_dir = os.path.join('..', 'data', 'clothing_color.v3i.multiclass', 'train')
relative_valid_dir = os.path.join('..', 'data', 'clothing_color.v3i.multiclass', 'valid')
relative_test_dir = os.path.join('..', 'data', 'clothing_color.v3i.multiclass', 'test')

train_dir = os.path.join(current_dir, relative_train_dir)
valid_dir = os.path.join(current_dir, relative_valid_dir)
test_dir = os.path.join(current_dir, relative_test_dir)

# Load your DataFrames
train_df = pd.read_csv(os.path.join(train_dir, '_classes.csv'))
valid_df = pd.read_csv(os.path.join(valid_dir, '_classes.csv'))
test_df = pd.read_csv(os.path.join(test_dir, '_classes.csv'))

# Strip leading spaces from column names
train_df.columns = train_df.columns.str.strip()
valid_df.columns = valid_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()

# Ensure that the 'colors' list also does not contain any leading spaces
colors = [color.strip() for color in colors]

def load_image(image_path):
    """Load an image from a file path and convert to numpy array."""
    img = load_img(image_path, target_size=(224, 224))  # Adjust the target size as needed
    img_array = img_to_array(img)
    return img_array

def load_images_and_labels(directory, classes_df, colors):
    """Load images and their corresponding labels from a directory and DataFrame."""
    images = []
    labels = []
    for index, row in classes_df.iterrows():
        image_path = os.path.join(directory, row['filename'])
        if os.path.exists(image_path):
            image = load_image(image_path)
            images.append(image)
            labels.append(row[colors].values.astype(np.float32))  # Ensure labels are float32
    return np.array(images), np.array(labels)

# Load training data
X_train, y_train = load_images_and_labels(train_dir, train_df, colors)

# Load validation data
X_val, y_val = load_images_and_labels(valid_dir, valid_df, colors)

# Load test data
X_test, y_test = load_images_and_labels(test_dir, test_df, colors)

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)
datagen.fit(X_train)

# Calculate class weights
class_counts = train_df[colors].sum()
total_samples = class_counts.sum()
class_weights = {i: total_samples / count for i, count in enumerate(class_counts)}

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3), kernel_regularizer='l2'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(64, (3, 3), activation='relu', kernel_regularizer='l2'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu', kernel_regularizer='l2'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu', kernel_regularizer='l2'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(len(colors), activation='sigmoid')  # Multi-label classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=50, 
                    validation_data=(X_val, y_val),
                    class_weight=class_weights,
                    callbacks=[reduce_lr, early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')


In [9]:
# model.save("dress_color_detector_task5.model",save_format="h5")